In [1]:
from misc import load, load_class_labels
from data_loader import data_loader;

dataset_root = "../dataset/"

train_dir = dataset_root + "train_set/"
val_dir = dataset_root + "val_set/"
val_degraded_dir = dataset_root + "val_set_degraded/"

sample_rate = 1
train_info = load(dataset_root + 'labels/train.csv', 1, sample_rate)
val_info = load(dataset_root + 'labels/validation.csv')

train_image_names = train_info.iloc[:, 0].values
val_image_names = val_info.iloc[:, 0].values

train_labels = train_info.iloc[:, 1].values
val_labels = val_info.iloc[:, 1].values
y_val = val_labels

class_labels = load_class_labels(dataset_root + 'classes.txt')

dl = data_loader(train_info, train_dir, 400, (224, 224));
dl_val = data_loader(val_info, train_dir, 400, (224, 224));


## Classification using handcrafted features

In [2]:
import numpy as np;
from handcrafted_extraction import extract_features

frac_of_train = 0.8

sift_info = {
    'max_number': 50,
    'num_of_sample_kmeans': 1,
    'voc_size': 300,
    'max_iter': 500
}

gabor_obj = {
    'angles': np.arange(0, np.pi, np.pi/4),
    'lambdas': np.arange(0, 1, 0.2),
    'gammas': [0.5]
}

color = True;
lbp_info = [2];

sampling = train_info.sample(frac=frac_of_train)
images_names = sampling.iloc[:, 0].values
y_train = sampling['label'].values

In [3]:
(train_features, kmeans) = extract_and_process_features(train_dir, images_names, kmeans=None, sift_info=sift_info, gabor_obj=gabor_obj, color=color, lbp_info=lbp_info)

0it [00:00, ?it/s]




1it [00:00,  9.80it/s]



5it [00:00, 23.77it/s]


95it [00:00, 268.03it/s]



15it [00:00, 37.28it/s]


20it [00:00, 40.76it/s]


25it [00:00, 43.53it/s]


30it [00:00, 44.55it/s]


35it [00:00, 45.59it/s]


40it [00:01, 42.79it/s]

46it [00:01, 43.63it/s]
51it [00:01, 42.55it/s]

56it [00:01, 41.18it/s]



61it [00:01, 38.32it/s]


65it [00:01, 37.65it/s]


70it [00:01, 39.86it/s]


75it [00:01, 36.83it/s]


79it [00:02, 35.64it/s]


84it [00:02, 38.56it/s]
89it [00:02, 41.15it/s]

94it [00:02, 42.34it/s]
95it [00:02, 39.67it/s]











95it [00:02, 34.10it/s]














95it [00:03, 26.21it/s]


Initialization complete
Iteration 0, inertia 445409024.0.
Iteration 1, inertia 312986816.0.
Iteration 2, inertia 306278400.0.
Iteration 3, inertia 303046624.0.
Iteration 4, inertia 301620928.0.
Iteration 5, inertia 300798976.0.
Iteration 6, inertia 300395904.0.
Iteration 7, inertia 300214944.0.
Iteration 8, inertia 300117664.0.
Iteration 9, inertia 300064256.0.
Converged at iteration 9: strict convergence.


## Standard classification

Common code for features extraction from validation set

In [4]:
(validation_features, kmeans) = extract_and_process_features(train_dir, val_image_names, kmeans=kmeans, sift_info=sift_info, gabor_obj=gabor_obj, color=color, lbp_info=lbp_info)

0it [00:00, ?it/s]


2it [00:00, 15.41it/s]


5it [00:00, 19.08it/s]


100it [00:00, 295.60it/s]

7it [00:00, 18.23it/s]




11it [00:00, 21.51it/s]


14it [00:00, 22.28it/s]


17it [00:00, 22.36it/s]


20it [00:00, 23.61it/s]


24it [00:01, 26.57it/s]


27it [00:01, 27.26it/s]

30it [00:01, 25.50it/s]


33it [00:01, 25.21it/s]


37it [00:01, 28.25it/s]


41it [00:01, 29.97it/s]





45it [00:01, 25.52it/s]


48it [00:01, 26.13it/s]


51it [00:02, 26.23it/s]


54it [00:02, 25.67it/s]


57it [00:02, 25.58it/s]


60it [00:02, 25.84it/s]
100it [00:02, 40.41it/s]
63it [00:02, 24.94it/s]
67it [00:02, 28.45it/s]
100it [00:02, 37.15it/s]
100it [00:03, 28.38it/s]


In [ ]:
train_and_val_features = train_features.copy();
train_and_val_features = np.concatenate((train_and_val_features, validation_features))

all_labels = train_labels.copy()
all_labels = np.concatenate((all_labels, val_labels))

### SVM

In [24]:
from sklearn import svm
from sklearn.model_selection import cross_validate

clf = svm.SVC(C=0.5)

Evaluation

In [35]:
scores = cross_validate(clf, train_and_val_features, all_labels, cv=5)

/home/gio/Documents/visual/Noisy-FoodX-251-Classification/.venv/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [36]:
print(scores)

{'fit_time': array([0.01554155, 0.01549411, 0.01541781, 0.01510024, 0.01501536]), 'score_time': array([0.00334811, 0.0030787 , 0.00326753, 0.0029242 , 0.0032208 ]), 'test_score': array([0.02564103, 0.02564103, 0.02564103, 0.02564103, 0.02564103])}


## Neural network classification

In [7]:
from tensorflow import keras

model = keras.Sequential([
    keras.Input(shape=390),
    keras.layers.Dense(200, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(251, activation='softmax')
])

model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'], run_eagerly=True)

2024-01-01 15:39:45.016779: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-01 15:39:45.016820: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-01 15:39:45.054597: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-01 15:39:45.126713: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-01 15:39:46.526696: W tensorflow/compiler/tf2

In [8]:
import pandas as pd
epoch_num = 40;

labels = keras.utils.to_categorical(train_labels.tolist(), num_classes=251)

for epoch in range(epoch_num):
    model.fit(train_features, labels, batch_size=100, shuffle=True, validation_split=0.1);

NameError: name 'all_features' is not defined

In [ ]:
val_labels = keras.utils.to_categorical(val_labels.tolist(), num_classes=251);
res = model.evaluate(validation_features, val_labels)